In [6]:
"""
The basis for a framework of automatic differentiation such as PyTorch.

What we want is to:
1. Build a forward pass manually, passing tensors / variable through blocks
2. Make sure each variable remember which block uses it (to get its gradients)
"""

import abc
import math
import numpy as np

In [7]:
class Variable:
    def __init__(self):
        self.gradient = []
    
        
    

In [10]:
import torch
x = torch.zeros((1, 1), requires_grad=True)
print(x)
y = torch.exp(x)
print(y)

tensor([[0.]], requires_grad=True)
tensor([[1.]], grad_fn=<ExpBackward>)
